# **Hunting Pemain Basket**

Usia (Age) <= 25 tahun,

Tinggi badan (Height) >= 180 cm,

Berat badan (Weight) <= 90 kg,

Rata-rata point (Average points scored) >= 6, dan

Rata-rata rebound (Average rebounds grabbed) >= 3.

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, balanced_accuracy_score, precision_score, f1_score

In [2]:
df = pd.read_csv('all_seasons.csv')
df.head()

,Unnamed: 0,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,gp,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,0,Chris Robinson,VAN,23.0,195.58,90.71840,Western Kentucky,USA,1996,2,51,41,4.6,1.7,1.6,-11.4,0.039,0.088,0.155,0.486,0.156,1996-97
1,1,Matt Fish,MIA,27.0,210.82,106.59412,North Carolina-Wilmington,USA,1992,2,50,6,0.3,0.8,0.0,-15.1,0.143,0.267,0.265,0.333,0.000,1996-97
2,2,Matt Bullard,HOU,30.0,208.28,106.59412,Iowa,USA,Undrafted,Undrafted,Undrafted,71,4.5,1.6,0.9,0.9,0.016,0.115,0.151,0.535,0.099,1996-97
3,3,Marty Conlon,BOS,29.0,210.82,111.13004,Providence,USA,Undrafted,Undrafted,Undrafted,74,7.8,4.4,1.4,-9.0,0.083,0.152,0.167,0.542,0.101,1996-97
4,4,Martin Muursepp,DAL,22.0,205.74,106.59412,None,USA,1996,1,25,42,3.7,1.6,0.5,-14.5,0.109,0.118,0.233,0.482,0.114,1996-97


In [3]:
recruit = []
for x in range(len(df.values)):
    if df['age'][x] <= 25 and df['player_height'][x] >= 180 and df['player_weight'][x] <= 90 and df['pts'][x] >= 6 and df['reb'][x] >= 3:
        recruit.append('lolos')
    else:
        recruit.append('tidak lolos')
df['recruit_criteria'] = recruit
df

,Unnamed: 0,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,gp,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season,recruit_criteria
0,0,Chris Robinson,VAN,23.0,195.58,90.71840,Western Kentucky,USA,1996,2,51,41,4.6,1.7,1.6,-11.4,0.039,0.088,0.155,0.486,0.156,1996-97,tidak lolos
1,1,Matt Fish,MIA,27.0,210.82,106.59412,North Carolina-Wilmington,USA,1992,2,50,6,0.3,0.8,0.0,-15.1,0.143,0.267,0.265,0.333,0.000,1996-97,tidak lolos
2,2,Matt Bullard,HOU,30.0,208.28,106.59412,Iowa,USA,Undrafted,Undrafted,Undrafted,71,4.5,1.6,0.9,0.9,0.016,0.115,0.151,0.535,0.099,1996-97,tidak lolos
3,3,Marty Conlon,BOS,29.0,210.82,111.13004,Providence,USA,Undrafted,Undrafted,Undrafted,74,7.8,4.4,1.4,-9.0,0.083,0.152,0.167,0.542,0.101,1996-97,tidak lolos
4,4,Martin Muursepp,DAL,22.0,205.74,106.59412,None,USA,1996,1,25,42,3.7,1.6,0.5,-14.5,0.109,0.118,0.233,0.482,0.114,1996-97,tidak lolos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9556,9556,Elijah Millsap,PHX,29.0,200.66,97.52228,Alabama-Birmingham,USA,Undrafted,Undrafted,Undrafted,2,1.5,3.0,0.5,-1.8,0.125,0.150,0.199,0.190,0.067,2016-17,tidak lolos
9557,9557,Elfrid Payton,ORL,23.0,193.04,83.91452,Louisiana-Lafayette,USA,2014,1,10,82,12.8,4.7,6.5,-2.7,0.039,0.136,0.213,0.520,0.329,2016-17,lolos
9558,9558,Edy Tavares,CLE,25.0,220.98,120.20188,None,Cabo Verde,2014,2,43,2,4.0,5.5,0.5,15.2,0.160,0.241,0.137,0.633,0.071,2016-17,tidak lolos
9559,9559,Willy Hernangomez,NYK,23.0,210.82,108.86208,None,Spain,2015,2,35,72,8.2,7.0,1.3,-2.8,0.142,0.263,0.202,0.564,0.116,2016-17,tidak lolos


In [4]:
df = df[['player_name', 'age', 'player_height', 'player_weight', 'pts', 'reb', 'recruit_criteria']]
df.head()

,player_name,age,player_height,player_weight,pts,reb,recruit_criteria
0,Chris Robinson,23.0,195.58,90.71840,4.6,1.7,tidak lolos
1,Matt Fish,27.0,210.82,106.59412,0.3,0.8,tidak lolos
2,Matt Bullard,30.0,208.28,106.59412,4.5,1.6,tidak lolos
3,Marty Conlon,29.0,210.82,111.13004,7.8,4.4,tidak lolos
4,Martin Muursepp,22.0,205.74,106.59412,3.7,1.6,tidak lolos


In [5]:
df.isnull().sum()

player_name         0
age                 0
player_height       0
player_weight       0
pts                 0
reb                 0
recruit_criteria    0
dtype: int64

In [6]:
prepro = df[['age', 'player_height', 'player_weight', 'pts', 'reb']]
prepro = prepro.rename(columns = {"age": "Age", "player_height": "Height", "player_weight": "Weight", "pts": "Avg Points", "reb": "Avg Rebounds"})
dfPre = pd.DataFrame(prepro)
dfPre = dfPre.astype(int)
dfPre.head()

,Age,Height,Weight,Avg Points,Avg Rebounds
0,23,195,90,4,1
1,27,210,106,0,0
2,30,208,106,4,1
3,29,210,111,7,4
4,22,205,106,3,1


In [7]:
scaler = StandardScaler()
scaler.fit(dfPre[["Age" ,"Height", "Weight", "Avg Points", "Avg Rebounds"]])
prePros = scaler.fit_transform(dfPre[["Age" ,"Height", "Weight", "Avg Points", "Avg Rebounds"]])
prePros

array([[-0.99674183, -0.58813059, -0.81730315, -0.61726116, -0.84600361],
       [-0.07727165,  1.02972208,  0.43267452, -1.29654684, -1.24475345],
       [ 0.61233098,  0.81400839,  0.43267452, -0.61726116, -0.84600361],
       ...,
       [-0.53700674,  2.10829052,  1.52640499, -0.61726116,  0.74899575],
       [-0.99674183,  1.02972208,  0.58892173,  0.06202452,  1.54649543],
       [ 0.61233098,  0.27472416,  0.1201801 ,  1.25077445,  1.14774559]])

In [8]:
dfPre['StdAge'] = prePros[:,0]
dfPre['StdHeight'] = prePros[:,1]
dfPre['StdWeight'] = prePros[:,2]
dfPre['StdAvg Points'] = prePros[:,3]
dfPre['StdAvg Rebounds'] = prePros[:, 4]
dfPre.head()

,Age,Height,Weight,Avg Points,Avg Rebounds,StdAge,StdHeight,StdWeight,StdAvg Points,StdAvg Rebounds
0,23,195,90,4,1,-0.996742,-0.588131,-0.817303,-0.617261,-0.846004
1,27,210,106,0,0,-0.077272,1.029722,0.432675,-1.296547,-1.244753
2,30,208,106,4,1,0.612331,0.814008,0.432675,-0.617261,-0.846004
3,29,210,111,7,4,0.382463,1.029722,0.823293,-0.107797,0.350246
4,22,205,106,3,1,-1.226609,0.490438,0.432675,-0.787083,-0.846004


#Baru

In [9]:
df2 = pd.read_excel('datapemain.xlsx')
df2

,Name,Club,Country,Age,Height,Weight,Avg Points,Avg Rebounds
0,Andakara Prastawa Dyaksa,Pelita Jaya Bakrie,Indonesia,24,190,90,7,6
1,Reggie Mononimbar,Pelita Jaya Bakrie,Indonesia,21,185,86,6,3
2,Hardianus Lakudu,Satria Muda Pertamina Jakarta,Indonesia,23,178,83,10,3
3,Kevin Yonas Sitorus,Satria Muda Pertamina Jakarta,Indonesia,26,185,75,11,4
4,Arki Dikania Wisnu,Satria Muda Pertamina Jakarta,Indonesia,20,183,80,5,2
5,Laurentius Steven Oei,Satria Muda Pertamina Jakarta,Indonesia,21,191,85,4,10
6,Mei Joni,Stapac,Indonesia,25,188,90,7,5
7,Vincent Rivaldi Kosasih,Stapac,Indonesia,23,179,87,1,2
8,Hardian Wicaksono,Pacific Caesar Surabaya,Indonesia,21,177,80,9,8
9,Brandon Jawato,Louvre Surabaya,Indonesia,24,182,85,6,5


In [10]:
df2 = df2[["Name", "Age", "Height", "Weight", "Avg Points", "Avg Rebounds"]]
df2

,Name,Age,Height,Weight,Avg Points,Avg Rebounds
0,Andakara Prastawa Dyaksa,24,190,90,7,6
1,Reggie Mononimbar,21,185,86,6,3
2,Hardianus Lakudu,23,178,83,10,3
3,Kevin Yonas Sitorus,26,185,75,11,4
4,Arki Dikania Wisnu,20,183,80,5,2
5,Laurentius Steven Oei,21,191,85,4,10
6,Mei Joni,25,188,90,7,5
7,Vincent Rivaldi Kosasih,23,179,87,1,2
8,Hardian Wicaksono,21,177,80,9,8
9,Brandon Jawato,24,182,85,6,5


In [11]:
scaler1 = StandardScaler()
scaler.fit(df2[["Age" ,"Height", "Weight", "Avg Points", "Avg Rebounds"]])
zz = scaler.fit_transform(df2[["Age" ,"Height", "Weight", "Avg Points", "Avg Rebounds"]])
zz

array([[ 0.63599873,  1.32911403,  1.31632171,  0.14285714,  0.48349378],
       [-0.95399809,  0.25724788,  0.42390021, -0.21428571, -0.72524067],
       [ 0.10599979, -1.24336474, -0.24541591,  1.21428571, -0.72524067],
       [ 1.69599661,  0.25724788, -2.0302589 ,  1.57142857, -0.32232919],
       [-1.48399703, -0.17149859, -0.91473203, -0.57142857, -1.12815215],
       [-0.95399809,  1.54348727,  0.20079484, -0.92857143,  2.09513971],
       [ 1.16599767,  0.90036757,  1.31632171,  0.14285714,  0.0805823 ],
       [ 0.10599979, -1.02899151,  0.64700558, -2.        , -1.12815215],
       [-0.95399809, -1.45773797, -0.91473203,  0.85714286,  1.28931674],
       [ 0.63599873, -0.38587182,  0.20079484, -0.21428571,  0.0805823 ]])

In [12]:
criteria = dfPre[(dfPre["Age"] <= 25) & (dfPre["Height"] >= 180) & (dfPre["Weight"] <= 90) & (dfPre["Avg Points"] >= 6) & (dfPre["Avg Rebounds"] >= 3)]
criteria.index.values

array([ 105,  290,  314,  432,  659,  835,  874, 1018, 1032, 1062, 1135,
       1141, 1332, 1351, 1436, 1515, 1516, 1720, 1727, 1748, 1875, 1951,
       2004, 2018, 2047, 2101, 2114, 2143, 2203, 2259, 2394, 2398, 2484,
       2544, 2551, 2616, 2631, 2704, 2836, 2840, 2873, 2984, 3010, 3081,
       3159, 3329, 3376, 3389, 3441, 3449, 3529, 3558, 3573, 3629, 3704,
       3717, 3721, 3830, 3994, 4036, 4039, 4068, 4121, 4124, 4252, 4267,
       4271, 4374, 4385, 4422, 4482, 4483, 4502, 4528, 4547, 4556, 4613,
       4624, 4633, 4665, 4762, 4775, 4782, 4800, 4924, 5015, 5070, 5154,
       5156, 5164, 5229, 5274, 5292, 5293, 5383, 5390, 5512, 5521, 5522,
       5548, 5587, 5609, 5675, 5743, 5747, 5820, 5914, 5952, 5954, 6001,
       6014, 6040, 6089, 6098, 6183, 6263, 6272, 6277, 6304, 6343, 6360,
       6408, 6409, 6441, 6450, 6542, 6555, 6639, 6710, 6752, 6810, 6822,
       6871, 6926, 6959, 6979, 7009, 7040, 7059, 7073, 7091, 7094, 7148,
       7184, 7199, 7305, 7306, 7342, 7365, 7369, 73

In [13]:
dfPre["Target"] = dfPre.apply(lambda dfPre : 1 if (dfPre["Age"] <= 25) and 
                                                (dfPre["Height"] >= 180) and  
                                                (dfPre["Weight"] <= 90) and  
                                                (dfPre["Avg Points"] >= 6) and  
                                                (dfPre["Avg Rebounds"] >= 3) else 0, axis = 1)
player = dfPre
player.head()

,Age,Height,Weight,Avg Points,Avg Rebounds,StdAge,StdHeight,StdWeight,StdAvg Points,StdAvg Rebounds,Target
0,23,195,90,4,1,-0.996742,-0.588131,-0.817303,-0.617261,-0.846004,0
1,27,210,106,0,0,-0.077272,1.029722,0.432675,-1.296547,-1.244753,0
2,30,208,106,4,1,0.612331,0.814008,0.432675,-0.617261,-0.846004,0
3,29,210,111,7,4,0.382463,1.029722,0.823293,-0.107797,0.350246,0
4,22,205,106,3,1,-1.226609,0.490438,0.432675,-0.787083,-0.846004,0


In [0]:
xtr,xts,ytr,yts = train_test_split(player[['Age','Height','Weight', "Avg Points", "Avg Rebounds"]], player['Target'], test_size = .18)

In [15]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver = "lbfgs")
model.fit(xtr,ytr)
model.score(xts,yts)

0.9837303893085415

In [16]:
penalty = ["l2"]
solver = ["newton-cg", "lbfgs", "liblinear", "sag", "saga"]
max_iter = [1, 10, 100, 1000, 10000]

param = {"penalty": penalty, "solver": solver, "max_iter" : max_iter}
model.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [17]:
from sklearn.model_selection import RandomizedSearchCV
model = LogisticRegression()
modelrs = RandomizedSearchCV(estimator = model, param_distributions = param, cv = 5)
modelrs.fit(xtr, ytr)

/usr/local/lib/python3.6/dist-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/optimize.py:204: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.6/d

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=LogisticRegression(C=1.0, class_weight=None,
                                                dual=False, fit_intercept=True,
                                                intercept_scaling=1,
                                                l1_ratio=None, max_iter=100,
                                                multi_class='auto', n_jobs=None,
                                                penalty='l2', random_state=None,
                                                solver='lbfgs', tol=0.0001,
                                                verbose=0, warm_start=False),
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'max_iter': [1, 10, 100, 1000, 10000],
                                        'penalty': ['l2'],
                                        'solver': ['newton-cg', 'lbfgs',
                                                   'liblinear', 'sag',

In [18]:
modelHPTLR = LogisticRegression(solver = "newton-cg", penalty = "l2", max_iter = 10000)
modelHPTLR.fit(xtr,ytr)
modelHPTLR.score(xts,yts)

0.9837303893085415

In [0]:
from sklearn.metrics import balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

In [20]:
model.fit(xtr, ytr)
ypred = model.predict(xts)
print(balanced_accuracy_score(yts, ypred))
print(precision_score(yts, ypred))
print(recall_score(yts, ypred))
print(f1_score(yts, ypred))
print(roc_auc_score(yts, ypred))
print(ypred[:10])
print(yts[:10])

0.6479302188054405
0.5625
0.3
0.3913043478260869
0.6479302188054407
[0 0 0 0 0 0 0 0 0 0]
5304    0
2782    0
380     0
9516    0
1338    0
8956    0
1680    0
2253    0
3403    0
1809    0
Name: Target, dtype: int64


In [21]:
xtr,xts,ytr,yts = train_test_split(player[['Age','Height','Weight', "Avg Points", "Avg Rebounds"]], player['Target'], test_size = .18)

from sklearn import tree

model1 = tree.DecisionTreeClassifier()
print(model1.fit(xtr,ytr))
print(model1.score(xts,yts))

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')
1.0


In [0]:
param2 = {"class_weight" : [None], 
          "criterion" : ['gini'], 
          "max_depth" : [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None] ,
          "max_features" : [None], 
          "max_leaf_nodes" : [None],
          "min_impurity_decrease" : [0.0], 
          "min_impurity_split" : [None],
          "min_samples_leaf" : [1], 
          "min_samples_split" : [2],
          "min_weight_fraction_leaf" : [0.0], 
          "presort" : [True, False],
          "random_state" : [None], 
          "splitter": ['best']}

In [23]:
modelrs = RandomizedSearchCV(estimator = model1, param_distributions = param2, cv = 5)
modelrs.fit(xtr, ytr)

/usr/local/lib/python3.6/dist-packages/sklearn/tree/_classes.py:319: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/tree/_classes.py:319: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/tree/_classes.py:319: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/tree/_classes.py:319: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress thi

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=None,
          

In [24]:
modelHPTDT = tree.DecisionTreeClassifier(splitter = "best", random_state = None, presort = False, min_weight_fraction_leaf = 0.0,
                                         min_samples_split = 2, min_samples_leaf = 1, min_impurity_split = None, min_impurity_decrease = 0.0,
                                         max_leaf_nodes = None, max_features = None, max_depth = 80, criterion = 'gini',
                                         class_weight = None)
print(modelHPTDT.fit(xtr,ytr))
print(modelHPTDT.score(xts,yts))

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=80, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')
1.0


/usr/local/lib/python3.6/dist-packages/sklearn/tree/_classes.py:319: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter.
  FutureWarning)


In [25]:
ypred = model1.predict(xts)
print(balanced_accuracy_score(yts, ypred))
print(precision_score(yts, ypred))
print(recall_score(yts, ypred))
print(f1_score(yts, ypred))
print(roc_auc_score(yts, ypred))
print(ypred[:10])
print(yts[:10])

1.0
1.0
1.0
1.0
1.0
[0 0 1 0 0 0 0 0 0 0]
2112    0
6868    0
3441    1
3458    0
1037    0
9494    0
9341    0
7855    0
4139    0
5027    0
Name: Target, dtype: int64


In [26]:
from sklearn.model_selection import train_test_split

xtr,xts,ytr,yts = train_test_split(player[['Age','Height','Weight', "Avg Points", "Avg Rebounds"]], player['Target'], test_size = .18)

from sklearn.ensemble import RandomForestClassifier

model2 = RandomForestClassifier(n_estimators = 100, max_depth = 2)
print(model2.fit(xtr,ytr))
print(model2.score(xts,yts))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=2, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
0.9761766414875073


In [27]:
param1 = {'bootstrap': [True, False],
          'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
          'max_features': ['auto', 'sqrt'],
          'min_samples_leaf': [1, 2, 4],
          'min_samples_split': [2, 5, 10],
          'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}
model2.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 2,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [28]:
modelHPTRF = RandomForestClassifier(n_estimators = 2000, min_samples_split = 10, min_samples_leaf = 1, max_features = "sqrt", max_depth = 100, bootstrap = "False")
print(modelHPTRF.fit(xtr,ytr))
print(modelHPTRF.score(xts,yts))

RandomForestClassifier(bootstrap='False', ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=100, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=2000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
0.999418942475305


In [29]:
ypred = model2.predict(xts)
print(balanced_accuracy_score(yts, ypred))
print(precision_score(yts, ypred))
print(recall_score(yts, ypred))
print(f1_score(yts, ypred))
print(roc_auc_score(yts, ypred))
print(ypred[:10])
print(yts[:10])

0.5
0.0
0.0
0.0
0.5
[0 0 0 0 0 0 0 0 0 0]
5001    0
4722    0
49      0
1684    0
5997    0
8242    0
8967    0
9090    0
60      0
6087    0
Name: Target, dtype: int64


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
df2["Recommendation"] = modelHPTDT.predict(df2[["Age", "Height", "Weight", "Avg Points", "Avg Rebounds"]])
df2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Name,Age,Height,Weight,Avg Points,Avg Rebounds,Recommendation
0,Andakara Prastawa Dyaksa,24,190,90,7,6,1
1,Reggie Mononimbar,21,185,86,6,3,1
2,Hardianus Lakudu,23,178,83,10,3,0
3,Kevin Yonas Sitorus,26,185,75,11,4,0
4,Arki Dikania Wisnu,20,183,80,5,2,0
5,Laurentius Steven Oei,21,191,85,4,10,0
6,Mei Joni,25,188,90,7,5,1
7,Vincent Rivaldi Kosasih,23,179,87,1,2,0
8,Hardian Wicaksono,21,177,80,9,8,0
9,Brandon Jawato,24,182,85,6,5,1
